# Завантаження та первинна обробка даних

Дані завантажено з Excel-файлу. Виконано перетворення колонок `RPM`, `Miles` і `Rate` у числовий формат, а також нормалізацію текстових полів `Broker` і `Status` — видалено зайві пробіли, а статуси приведено до верхнього регістру для уніфікації.

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.read_excel("../Loads 2024.xlsx", sheet_name="DATA")

df["RPM"] = pd.to_numeric(df["RPM"], errors="coerce")
df["Miles"] = pd.to_numeric(df["Miles"], errors="coerce")
df["Rate"] = pd.to_numeric(df["Rate"], errors="coerce")
df["Broker"] = df["Broker"].astype(str).str.strip()
df["Status"] = df["Status"].astype(str).str.strip().str.upper()

# Відбір записів для розрахунку середнього RPM та стабільності

Вибрано лише вантажі зі статусом `BILLED` та позитивним значенням пройдених миль. Для них розраховано середній RPM (дохід за милю) та стандартне відхилення RPM по кожному брокеру, що відображає стабільність ціноутворення.

In [2]:
df_rpm = df[(df["Status"] == "BILLED") & (df["Miles"] > 0)]

avg_rpm = df_rpm.groupby("Broker")["RPM"].mean()
std_rpm = df_rpm.groupby("Broker")["RPM"].std().fillna(0)

# Обчислення рівня успішності

Для брокерів розраховано частку вантажів зі статусом `BILLED` від загальної кількості вантажів зі статусом `BILLED` або `CANCELED`. Цей показник відображає надійність брокера у виконанні замовлень.

In [3]:
df_success = df[df["Status"].isin(["BILLED", "CANCELED"])]
success_counts = df_success.groupby(["Broker", "Status"]).size().unstack(fill_value=0)
success_rate = success_counts["BILLED"] / (success_counts["BILLED"] + success_counts["CANCELED"])

# Розрахунок загального обсягу вантажів

Підраховано загальну кількість вантажів, оброблених кожним брокером, незалежно від статусу. Цей показник відображає масштаб діяльності брокера.

In [4]:
volume = df.groupby("Broker").size()

# Формування основної таблиці метрик

Об'єднано усі розраховані показники (середній RPM, стабільність, рівень успішності, обсяг вантажів) в єдиний DataFrame для подальшого аналізу. Пропущені значення заповнено нулями.

In [5]:
metrics = pd.DataFrame({
    "Avg RPM": avg_rpm,
    "RPM Std": std_rpm,
    "Success Rate": success_rate,
    "Volume": volume
}).fillna(0)

# Нормалізація метрик

Використано `MinMaxScaler` для масштабування трьох основних метрик (`Avg RPM`, `Success Rate`, `Volume`) до діапазону [0,1]. Для стабільності (`RPM Std`), де менше значення — краще, застосовано інверсію, щоб більші нормалізовані значення відповідали кращій стабільності.

In [6]:
scaler = MinMaxScaler()

metrics[["Norm RPM", "Norm Success", "Norm Volume"]] = scaler.fit_transform(
    metrics[["Avg RPM", "Success Rate", "Volume"]]
)

rpm_std_scaled = scaler.fit_transform(metrics[["RPM Std"]])
metrics["Norm Stability"] = 1 - rpm_std_scaled.flatten()

# Обчислення зваженого скору "крутості" брокера

Зважено поєднано нормалізовані метрики з заданими коефіцієнтами важливості:

- `Norm RPM` — 40% (прибутковість)
- `Norm Success` — 30% (надійність)
- `Norm Volume` — 20% (масштаб)
- `Norm Stability` — 10% (стабільність ціноутворення)

Отримано загальний скор, який потім масштабовано до шкали від 1 до 10.

In [7]:
weights = {
    "Norm RPM": 0.4,
    "Norm Success": 0.3,
    "Norm Volume": 0.2,
    "Norm Stability": 0.1
}

metrics["Score"] = (
    metrics["Norm RPM"] * weights["Norm RPM"] +
    metrics["Norm Success"] * weights["Norm Success"] +
    metrics["Norm Volume"] * weights["Norm Volume"] +
    metrics["Norm Stability"] * weights["Norm Stability"]
)

metrics["Score_1_10"] = metrics["Score"] * 9 + 1

# Формування фінального рейтингу брокерів

Отримано відсортований за зменшенням скору DataFrame з метриками та рейтингом брокерів для подальшого аналізу та прийняття рішень.

In [8]:
top_brokers = metrics.sort_values("Score_1_10", ascending=False)

top_brokers[["Avg RPM", "Success Rate", "Volume", "RPM Std", "Score_1_10"]].head(10)

,Avg RPM,Success Rate,Volume,RPM Std,Score_1_10
Broker,,,,,
ALEXANDRA FOODS,57.692300,1.000000,2,59.832123,7.303125
COYOTE,1.806544,1.000000,577,0.959339,6.498298
ARRIVE LOG,1.831022,1.000000,421,0.791027,6.014857
MOON EXPRESS,21.875000,1.000000,2,0.000000,5.968125
CHRW,1.885811,0.997126,385,0.647634,5.900174
ECHO,2.803470,0.997763,479,28.431655,5.834975
UBER FREIGHT,1.817245,0.996094,281,0.486198,5.570536
LINEAGE TRANS,3.036542,1.000000,244,0.701032,5.538310
SYNERGY,14.179100,1.000000,2,0.000000,5.487901
